# Exercise on supervised learning

In this exercise, you will train machine learning methods to predict if a person has a heart disease.

The goal is to learn:
*	how to apply different ML classification methods.
*	how to perform cross-validation.
*	how to assess and visualize the performance.
*	how to get an idea on which features are most important.

You will be using the same titantic dataset as you worked with on Monday, but instead of visualizing it and calculating properties you will be training ML methods to predict the chance of survival. You will train different ML methods using different parameters and different features using cross-validation to evaluate performance. For each ML method, you should save your final model, so you can load it and use it for predicting on new data later. Once you have trained and optimized a few ML methods, you will be given a new independent test set to finally check your performance on completely unseen data. We will compare which ML methods performed best, what was the cross-validated performance and what was the test set performance. Hopefully we will choose slightly different methods and training parameters, so we can compare what seems to work best.

We start by reading in the data....


In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#This is just some code to split the data and get the 'secret' test set as 20%
#data=pd.read_csv('heart_all_data.csv')
#mask = np.random.rand(len(data)) < 0.8
#train = data[mask]
#test = data[~mask]
#train.to_csv('heart_train.csv',index=False)
#test.to_csv('heart_test.csv',index=False)

#Read in the data to a pandas DataFrame using the read_csv method.
train=pd.read_csv('heart_train.csv')

# We are using the train data as placeholder for the test data,
# so we can implement code to run predictions on the test pandas DataFrame as well.
test=train.copy() 

#Uncomment this line to read in the true test, it will be revealed in due time....
#test=pd.read_csv('heart_test.csv')




: 

# Description of columns

- **age**: The person's age in years
- **sex**: The person's sex (1 = male, 0 = female)
- **cp:** The chest pain experienced (Value 1: typical angina, Value 2: atypical angina, Value 3: non-anginal pain, Value 4: asymptomatic)
- **trestbps:** The person's resting blood pressure (mm Hg on admission to the hospital)
- **chol:** The person's cholesterol measurement in mg/dl
- **fbs:** The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false) 
- **restecg:** Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)
- **thalach:** The person's maximum heart rate achieved
- **exang:** Exercise induced angina (1 = yes; 0 = no)
- **oldpeak:** ST depression induced by exercise relative to rest ('ST' relates to positions on the ECG plot. See more [here](https://litfl.com/st-segment-ecg-library/))
- **slope:** the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3: downsloping)
- **ca:** The number of major vessels (0-3)
- **thal:** A blood disorder called thalassemia (1 = normal; 2 = fixed defect; 3 = reversable defect)
- **target:** Heart disease (0 = no, 1 = yes)

The column names are abbreviations, we can change the column names by providing a list of equal size with the new names, like so:



Let's rename the columns and at categorize as we did on monday.



In [ ]:
# a function to do the categorization
def categorize(df):
    df.loc[df['sex']==0,'sex']='female'
    df.loc[df['sex']==1,'sex']='male'
    df.loc[df['chest_pain_type'] == 1,'chest_pain_type'] = 'typical angina'
    df.loc[df['chest_pain_type'] == 2,'chest_pain_type'] = 'atypical angina'
    df.loc[df['chest_pain_type'] == 3,'chest_pain_type'] = 'non-anginal pain'
    df.loc[df['chest_pain_type'] == 4,'chest_pain_type'] = 'asymptomatic'

    df.loc[df['fasting_blood_sugar'] == 0,'fasting_blood_sugar'] = 'lower than 120mg/ml'
    df.loc[df['fasting_blood_sugar'] == 1,'fasting_blood_sugar'] = 'greater than 120mg/ml'

    df.loc[df['rest_ecg'] == 0,'rest_ecg'] = 'normal'
    df.loc[df['rest_ecg'] == 1,'rest_ecg'] = 'ST-T wave abnormality'
    df.loc[df['rest_ecg'] == 2,'rest_ecg'] = 'left ventricular hypertrophy'

    df.loc[df['exercise_induced_angina'] == 0,'exercise_induced_angina'] = 'no'
    df.loc[df['exercise_induced_angina'] == 1,'exercise_induced_angina'] = 'yes'

    df.loc[df['st_slope'] == 1,'st_slope'] = 'upsloping'
    df.loc[df['st_slope'] == 2,'st_slope'] = 'flat'
    df.loc[df['st_slope'] == 3,'st_slope'] = 'downsloping'

    df.loc[df['thalassemia'] == 1,'thalassemia'] = 'normal'
    df.loc[df['thalassemia'] == 2,'thalassemia'] = 'fixed defect'
    df.loc[df['thalassemia'] == 3,'thalassemia'] = 'reversable defect'


In [ ]:
columns = ['age', 'sex', 'chest_pain_type', 
                 'resting_blood_pressure', 'cholesterol', 
                 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate',
       'exercise_induced_angina', 'st_depression', 'st_slope', 
                 'num_major_vessels', 'thalassemia', 'target']

train.columns=columns
test.columns=columns

#make a copy before changing.
train_orig=train.copy()
test_orig=test.copy()
categorize(train)
categorize(test)
train

Let's also convert them to dummy variables:

In [ ]:
df_train=pd.get_dummies(train)
df_test=pd.get_dummies(test)
df_train

## Cross-validation sets
Cross-validation is a way to estimate performance by training and testing on subsets of the data. Usually it is common to divide the data into 5 parts (with no similarity between them). If there is no similarity between the data points, the division can be random. But usually there is some connection between data points. In the case of training on sequence data, sequence similarity is commonly used to group data points. 

In this case it is OK to do cross-validation randomly



## Scaling
For some machine learning methods, e.g. SVM, scaling the features between 0-1 can improve the convergence and performance. Use the same `preprocessing.MinMaxScalar()` from `sklearn` as we used on Monday. If you want you can choose to compare the performance of scaling vs. no scaling. 

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

df=df_train
#Pick the columns that have numbers that can be used for training:
columns_to_scale=[c for c in df.columns if c !='target']

#Make train_data and test_data to be used for training and testing.
#We are using the one with the dummy variable, one-hot encoded.
train_data=df_train.copy()
test_data=df_test.copy()


scaling=True
#Uncomment to turn on scaling
#scaling=True
if scaling:
    # Fit the scaler on the training data
    # OBS If you do scaling you have to save 
    # the scaling parameters from the training set you know how to scale unknown examples in the prediction face.
    min_max_scaler.fit(train_data[columns_to_scale].values)
    # Transform the scaling to the train_data
    train_data.loc[:,columns_to_scale]=min_max_scaler.transform(train_data[columns_to_scale].values)
    # Transform the scaling to the test_data
    test_data.loc[:,columns_to_scale]=min_max_scaler.transform(test_data[columns_to_scale].values)



## Setting up the data for training.
Set up the data by making a numpy matrix of the training data called `X`, numpy vector with the target values `Y`, and if you are using cross-validitation use the PreDefinedSplit class (http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.PredefinedSplit.html) to


In [ ]:
from sklearn.model_selection import PredefinedSplit

#Put the training data in X the .values method returns a numpy matrix of the numbers in the DataFrame.
trainable_cols=[a for a in train_data.columns if a!='target']
X=train_data[trainable_cols].values #,0:target_index]

#Create the same for the test (currently this is the same as train just to keep to code running)
X_test=test_data[trainable_cols].values

#Put the target value in Y
Y=train_data['target'].values
Y_test=test_data['target'].values

#Use the PredefinedSplit class to define the cross-validation sets.
#Here it is random, but it could be based on some initial clustering of data. 
#Like sequence similarity, same 'patient', same/similar 'something'
folds=5
np.random.seed(42) #for reproducibility
cv = PredefinedSplit((np.random.rand(len(train_data))*folds).astype(int))


## Training Machine Learning Methods
Finally we can train machine learning methods... 

In [ ]:
#Import the functions
import sklearn
import numpy as np
from sklearn.model_selection import cross_val_score

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

#Define a function to calculate performance.
def calc_performance(correct,pred,pred_prob,name='',plot=False):
    acc=accuracy_score(correct,pred)
    mcc=matthews_corrcoef(correct,pred)
    f1=f1_score(correct,pred)
    if plot:
        (prec,recall,thres)=precision_recall_curve(correct,pred_prob)
        (fpr,tpr,thres_roc)=roc_curve(correct,pred_prob)   
        plt.plot(fpr,tpr)
        plt.title(f'ROC curve {name}')
        plt.xlabel('fpr')
        plt.ylabel('tpr')
        plt.show()
        plt.plot(prec,recall)
        plt.title(f'Prec-Recall curve {name}')
        plt.xlabel('Prec')
        plt.ylabel('Recall')
        plt.show()
    return(acc,mcc,f1)



#define a function to do the training using predefined splits

def train_ml(clf,X,Y,cv,verbose=True,plot=False,name=''):

    print(clf)
    #Some lists to store cross-validated predictions
    pred_save=[]
    true_save=[]
    pred_prob_save=[]

    for i, (train_index, val_index) in enumerate(cv.split(),1):
        if verbose:
            print("Set: ",i)
            print("Training on",len(train_index),"examples")
            print("Validating on",len(val_index),"examples")
        (X_train, X_val) = X[train_index,:], X[val_index,:]
        (Y_train, Y_val) = Y[train_index], Y[val_index]
  
        #Perform the training (fitting)
        clf=clf.fit(X_train,Y_train)
       
        #Predict on the training data    
        pred=clf.predict(X_train) #gives 0/1
        try:
            pred_prob=clf.predict_proba(X_train)[:,1] #gives a probability used in Roc plots.
        except:
            pred_prob=pred
        #Calculate performance measures on the training data
        (acc_train,mcc_train,f1_train)=calc_performance(pred,Y_train,pred_prob,plot=False)

    
        #Predict on the validation data
        
        val_pred=clf.predict(X_val)      
        try:
            val_pred_prob=clf.predict_proba(X_val)[:,1]
        except:
            val_pred_prob=val_pred
        #Save the values to have predictions for all folds.
        pred_save.append(val_pred)
        pred_prob_save.append(val_pred_prob)
        true_save.append(Y_val)
        #Calculate performance measures on the validation data
        (acc,mcc,f1)=calc_performance(Y_val,val_pred,val_pred_prob,plot=False)
        if verbose:
            print("Training performance","f1:",f1_train,"acc:",acc_train,"mcc:",mcc_train)
            print("Validation performance","f1:",f1,"acc:",acc,"mcc:",mcc)
            print("==============")

     
   

    #Concatenate the predictions for all folds to one vector
    predictions=np.concatenate(pred_save)
    correct=np.concatenate(true_save)
    predicted_prob=np.concatenate(pred_prob_save)

    #Calculate overall validation performance
    if verbose:
        print(f'ML Algorithm: {clf}')
    (acc,mcc,f1)=calc_performance(correct,predictions,predicted_prob,plot=plot,name=name)
    print(f"Overall Validation Performance f1 {f1} acc: {acc} mcc: {mcc}")
    #returns the correct, predictions, and predicted_prob for the cross-validation.
    #the inputed clf is a reference and will change.
    return(correct,predictions,predicted_prob)

In [ ]:
from sklearn import svm
from sklearn import linear_model
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#Define a ML algorithm to use.
#Feel free to check the sci-kit learn docs for the different algorithms to use
#and the specific options and details on the hyperparameters.
#Make sure you are are using method for Classification and not Regression



#Go back and change the trainable features / scaling, trying to figure out which method to use.
#Which method gives best performance on the validation data?
#What are the minimal number of features you need and still reach good performance on validation?

#clf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, n_jobs=4,random_state=None,verbose=0)
#clf = svm.SVC(kernel='linear',verbose=1, probability=True)
#clf = linear_model.LogisticRegression(max_iter=1000)
#clf = SGDClassifier(loss="hinge", penalty="l2")
#clf = KNeighborsClassifier(n_neighbors=5, weights='uniform')
#clf = KNeighborsClassifier(n_neighbors=5, weights='distance')

clf_log = linear_model.LogisticRegression(max_iter=1000)
(correct,predictions,predicted_prob)=train_ml(clf_log,X,Y,cv,verbose=False,plot=False)

In [ ]:
clf = linear_model.LogisticRegression(max_iter=1000)
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)

In [ ]:
clf = DecisionTreeClassifier(max_depth=None)
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)

(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
(prec,recall,thres)=precision_recall_curve(correct,predicted_prob)
acc=accuracy_score(correct,predictions)
mcc=matthews_corrcoef(correct,predictions)
f1=f1_score(correct,predictions) 
#plt.clf()
#plt.plot(thres_roc,fpr)
#plt.plot(thres_roc,tpr)
plt.plot(fpr,tpr,label='DecisionTree')
plt.xlabel('fpr')
plt.ylabel('tpr')

In [ ]:
clf = DecisionTreeClassifier(max_depth=5)
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)

(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
plt.plot(fpr,tpr,label='DecisionTree5')

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5, weights='uniform')
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)   
plt.plot(fpr,tpr,label='KN-uniform')
plt.legend()

In [ ]:
clf = SGDClassifier(loss="hinge", penalty="l2")
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)   
plt.plot(fpr,tpr,label='SGD')

In [ ]:
clf = svm.SVC(kernel='rbf',C=0.1) #I had problems using kernel='linear' got stuck in a loop /BW
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)   
plt.plot(fpr,tpr,label='SVMrbf')
plt.legend()

clf = svm.LinearSVC(max_iter=1000)
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)   
plt.plot(fpr,tpr,label='SVMlinear')
plt.legend()


clf = KNeighborsClassifier(n_neighbors=5, weights='uniform')
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)   
plt.plot(fpr,tpr,label='KN-uniform')
plt.legend()

clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)   
plt.plot(fpr,tpr,label='KN-distance')
plt.legend()



#clf = KNeighborsClassifier(n_neighbors=5, weights='distance')

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, n_jobs=4,random_state=None,verbose=0)
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
plt.plot(fpr,tpr,label='RF_no_MAX')
plt.xlabel('fpr')
plt.ylabel('tpr')

clf_RF10 = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, n_jobs=4,random_state=None,verbose=0)
(correct,predictions,predicted_prob)=train_ml(clf_RF10,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
plt.plot(fpr,tpr,label='RF_10')

clf_RF = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2, n_jobs=4,random_state=None,verbose=0)
(correct,predictions,predicted_prob)=train_ml(clf_RF,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
plt.plot(fpr,tpr,label='RF_5')


clf = RandomForestClassifier(n_estimators=100, max_depth=3, min_samples_split=2, n_jobs=4,random_state=None,verbose=0)
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
plt.plot(fpr,tpr,label='RF_3')

plt.legend()

In [ ]:
clf = DecisionTreeClassifier(max_depth=None)
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
plt.plot(fpr,tpr,label='DecisionTree')
plt.xlabel('fpr')
plt.ylabel('tpr')

clf = DecisionTreeClassifier(max_depth=5)
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
plt.plot(fpr,tpr,label='DecisionTree5')
plt.legend()


clf = DecisionTreeClassifier(max_depth=10)
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
plt.plot(fpr,tpr,label='DecisionTree10')
plt.legend()


clf = DecisionTreeClassifier(max_depth=2)
(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
plt.plot(fpr,tpr,label='DecisionTree2')
plt.legend()

#clf = DecisionTreeClassifier(max_depth=5,splitter='random')
#(correct,predictions,predicted_prob)=train_ml(clf,X,Y,cv,verbose=False,plot=False)
#(fpr,tpr,thres_roc)=roc_curve(correct,predicted_prob)
#plt.plot(fpr,tpr,label='DecisionTree5-rand')
#plt.legend()


In [ ]:
#Predict on test (this is identical to the train set until the true test is revealed.)
#Assumes the optimized ML method is defined as clf
clf=clf_log


test_correct=Y_test
test_pred=clf.predict(X_test)      
test_pred_prob=clf.predict_proba(X_test) 

(fpr,tpr,thres_roc)=roc_curve(test_correct,test_pred_prob[:,1])
plt.plot(fpr,tpr,label='logistic')
(acc,mcc,f1)=calc_performance(test_correct,test_pred,test_pred_prob[:,1],plot=True)
print("Overall Test Performance","f1:",f1,"acc:",acc,"mcc:",mcc)

In [ ]:
clf=clf_RF10
test_correct=Y_test
test_pred=clf.predict(X_test)      
test_pred_prob=clf.predict_proba(X_test) 

#(fpr,tpr,thres_roc)=roc_curve(test_correct,test_pred_prob[:,1])
#plt.plot(fpr,tpr,label='RF')
#plt.legend()
(acc,mcc,f1)=calc_performance(test_correct,test_pred,test_pred_prob[:,1],plot=True)
print("Overall Test Performance","f1:",f1,"acc:",acc,"mcc:",mcc)

(prec,recall,thres)=precision_recall_curve(test_correct,test_pred_prob[:,1])
plt.plot(prec,recall,label='RF')
#len(thres)
#len(prec)
plt.show()
plt.plot(thres,prec[1:],label='prec')
plt.plot(thres,recall[1:],label='recall')
plt.legend()
plt.show()


In [ ]:
#If you are using decision trees you check the importance of each feature
#it will complain if 'clf' does not have the 'feature_importances_' method.

try:
    for feature, importance in zip(trainable_cols,clf_RF.feature_importances_):
        print(f'{feature:18s} {importance:.3f}')
except:
    print('Probably clf does not have feature_importances_ method')



